In [13]:
import redis

r = redis.Redis(host='127.0.0.1', port=6379, db=0)
r

Redis<ConnectionPool<Connection<host=127.0.0.1,port=6379,db=0>>>

Завдання

На основі базових структур Redis змоделюйте  Shopping cart, де зберігається інформація про замовлення користувачів.
Shopping cart має включати:
- ідентифікатор замовлення
- інформацію про замовника (прізвище, ім’я, логін)
- товари в замовленні та їх кількість, 
- вартість замовлення,
- статус замовлення (In Progress -> Waiting for payment -> Paid | Canceled).

По замовленню необхідно мати можливість робити наступні запити:
Замовлення по Id
Всі ідентифікатори замовлення статус яких Waiting for payment
Всі ідентифікатори замовлення певного замовника
Всі ідентифікатори замовлення певного замовника з можливістю вказати статус для пошуку


In [49]:
order_status = {"In Progress": "1", "Waiting for payment": "2", "Paid": "3", "Canceled": "4"}

order_1 = {"id": 1, "client_name": "Ivan", "client_surname": "Popov", "login": "ipopov", "price": "500", "status": "2"}
order_2 = {"id": 2, "client_name": "Victor", 
           "client_surname": "Fedorov", "login": "vfedorov", "price": "100", "status": "2"}
r.hset('order:1', mapping=order_1)
r.hset('order:2', mapping=order_2)

# key is product_id, value is qty
products_in_order_1 = {"1": 2, "2": 2}
products_in_order_2 = {"4": 10}
r.hset("order:1:products", mapping=products_in_order_1)
r.hset("order:2:products", mapping=products_in_order_2)

waiting_orders = [1, 2] 
r.sadd('orders_by_status:2', *waiting_orders)

r.sadd('orders_by_client:ipopov', 1)
r.sadd('orders_by_client:vfedorov', 2)


r.sadd('orders_by_client:ipopov:' + order_status["Waiting for payment"], 1)
r.sadd('orders_by_client:vfedorov:' + order_status["Waiting for payment"], 2)

1

In [33]:
# Замовлення по Id
order_id = str(1)
r.hgetall("order:" + order_id)

{b'client_name': b'Ivan',
 b'client_surname': b'Popov',
 b'login': b'ipopov',
 b'price': b'500',
 b'status': b'1'}

In [48]:
# Всі ідентифікатори замовлення статус яких Waiting for payment
r.smembers('orders_by_status:' + order_status["Waiting for payment"])

{b'1', b'2'}

In [44]:
# Всі ідентифікатори замовлення певного замовника
r.smembers('orders_by_client:' + 'ipopov')

{b'1'}

In [51]:
# Всі ідентифікатори замовлення певного замовника з можливістю вказати статус для пошуку
r.smembers('orders_by_client:' + 'ipopov' ":" + order_status["Waiting for payment"])

{b'1'}

In [52]:
r.keys("*")

[b'orders_by_client:ipopov:2',
 b'order:1:products',
 b'order:2:products',
 b'order_status',
 b'order:2',
 b'orders_by_client:ipopov',
 b'foo:test',
 b'orders_by_status:2',
 b'orders_by_client:vfedorov:2',
 b'orders_by_client:vfedorov',
 b'foo',
 b'foo:test2',
 b'order:1']